<a href="https://colab.research.google.com/github/cyyeh/ds-portfolios/blob/master/rapids_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation Guide on cuDF, cuML

## Environment Sanity Check

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [1]:
!nvidia-smi

Sun Feb  9 23:09:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [3]:
!python -V; pip -V

Python 3.6.9
pip 19.3.1 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


## Install cuDF and make some adjustment in order to make it work.

installation instructions: https://gist.github.com/AK-ayush/a0d2f389bc013c3e2345c268034ce78d

In [4]:
!pip install cudf-cuda100

     |████████████████████████████████| 17.2MB 201kB/s 
     |████████████████████████████████| 12.4MB 30.1MB/s 
     |████████████████████████████████| 3.2MB 56.9MB/s 
     |████████████████████████████████| 9.1MB 10.3MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Found existing installation: numba 0.47.0
    Uninstalling numba-0.47.0:
      Successfully uninstalled numba-0.47.0


In [1]:
!find / -name librmm.so*

/usr/local/lib/python3.6/dist-packages/librmm.so


In [0]:
!cp /usr/local/lib/python3.6/dist-packages/librmm.so . 

In [0]:
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

### [cuDF](https://github.com/rapidsai/cudf)

_Note_: You must import nvstrings and nvcategory before cudf, else you'll get errors.

In [4]:
import nvstrings, nvcategory, cudf
import io, requests

# download CSV file from GitHub
url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

# read CSV from memory
tips_df = cudf.read_csv(io.StringIO(content))
tips_df['tip_percentage'] = tips_df['tip']/tips_df['total_bill']*100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

size
1     21.72920154872781
2    16.571919173482897
3    15.215685473711831
4    14.594900639351334
5    14.149548965142023
6    15.622920072028379
Name: tip_percentage, dtype: float64


## Install cuML and make some adjustment in order to make it work.

installation instructions: https://gist.github.com/AK-ayush/c9b81a25062ddb1616e6f6d8ff767093

In [5]:
!apt install libopenblas-base libomp-dev
!pip install cuml-cuda100

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-base is already the newest version (0.2.20+ds-4).
libopenblas-base set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (236 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 145113 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb .

In [6]:
!find / -name libcuml.so*

/usr/local/lib/python3.6/dist-packages/libcuml.so


In [0]:
!cp /usr/local/lib/python3.6/dist-packages/libcuml.so /usr/lib64-nvidia/

### [cuML](https://github.com/rapidsai/cuml)

In [8]:
import cuml

# Create and populate a GPU DataFrame
df_float = cudf.DataFrame()
df_float['0'] = [1.0, 2.0, 5.0]
df_float['1'] = [4.0, 2.0, 1.0]
df_float['2'] = [4.0, 2.0, 1.0]

# Setup and fit clusters
dbscan_float = cuml.DBSCAN(eps=1.0, min_samples=1)
dbscan_float.fit(df_float)

print(dbscan_float.labels_)

0    0
1    1
2    2
dtype: int32


## Next Steps

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-extended